In [ ]:
import numpy as np

In [ ]:
class GridWorld:

  def __init__(self, rows, cols, start, goal):

    self.env = np.zeros((rows, cols))

    self.start = start

    self.goal = goal

    self.env[goal] = 0

    self.env[start] = 0

    # Rmat[0:n-1,1,:],Rmat[1:,3,:] = -100,-100

    # self.


    self.rows = rows

    self.cols = cols

    self.cur_values = np.zeros((rows,cols))

    self.qvalues = np.random.uniform(0,1,(4,rows,cols))

    self.nstates = np.zeros((4,rows,cols))

    


  def nextState(self, s, a):

    if a == 0:

      if s[1] == 0:
        return (s[0],s[1])

      return (s[0], s[1] - 1)

    elif a == 1:

      if s[1] == self.cols - 1:
        return (s[0], s[1])

      return (s[0], s[1] + 1)

    elif a == 2:

      if s[0] == 0:
        return (s[0], s[1])

      return (s[0] - 1, s[1])

    elif a == 3:

      if s[0] == self.rows - 1:
        return (s[0], s[1])

      return (s[0] + 1, s[1])

  def reward(self, s, a):

    if a == 0:

      if s[1] == 0:
        # return self.env[(s[0],s[1])] - 1
        return -1


      # return self.env[(s[0], s[1] - 1)]
      return -1

    elif a == 1:

      if s[1] == self.cols - 1:
        # return self.env[(s[0], s[1])] - 1
        return -1


      # return self.env[(s[0], s[1] + 1)]
      return -1

    elif a == 2:

      if s[0] == 0:
        return -1
        # return self.env[(s[0], s[1])] - 1

      # return self.env[(s[0] - 1, s[1])]
      return -1

    elif a == 3:

      if s[0] == self.rows - 1:
        # return self.env[(s[0], s[1])] - 1
        return -1


      # return self.env[(s[0] + 1, s[1])]
      return -1

    


In [ ]:
class Agent:

  def __init__(self, rows, cols, start):

    self.actions = ["left", "right", "up", "down"]

    self.policy_values = np.full((4,rows,cols),0.25)

    self.start = start

    self.cur_state = start

    self.epsilon = np.random.uniform(0,1)




In [ ]:
class PolicyIteration:

  def __init__(self, agent, env):

    self.df = 1

    self.step_size = np.random.uniform(0,1)
    
    self.agent = agent

    self.env = env

    self.epno = 0

  def evaluate_policy(self):

    s = [np.random.randint(0,self.env.rows), np.random.randint(0,self.env.cols)]

    self.env.nstates = np.zeros((4,self.env.rows,self.env.cols))

    ret = 0

    count = 0
    
    self.epno += 1

    self.agent.epsilon = 1/self.epno

    while s != self.env.goal:

      a = np.random.choice([0,1,2,3] ,p=self.agent.policy_values[:,s[0],s[1]])
      
      s1 = self.env.nextState(s,a)

      ret += (self.df ** count) * self.env.reward(s,a)

      self.env.nstates[a,s[0],s[1]] += 1

      count += 1

      self.env.qvalues[a,s[0],s[1]] += (1/self.env.nstates[a,s[0],s[1]]) * (ret - self.env.qvalues[a,s[0],s[1]])

      s = s1

      if(count == 100):
        break


    
  


  def findMax(self,s):

    return self.env.qvalues[:,s[0],s[1]]

    # if s[0] >= 0 and s[1] - 1 >= 0:

    #   vals.append(self.env.qvalues[a,s[0],s[1]])

    # else:

    #   vals.append(self.env.cur_values[s[0],s[1]])
    #   # vals.append(0)


    # if s[0] >=0 and s[1] + 1 <= self.env.cols - 1:

    #   vals.append(self.env.cur_values[s[0],s[1]+1])

    # else:

    #   vals.append(self.env.cur_values[s[0],s[1]])
    #   # vals.append(0)



    # if s[0] - 1 >= 0 and s[1] >= 0:

    #   vals.append(self.env.cur_values[s[0] - 1,s[1]])

    # else:

    #   vals.append(self.env.cur_values[s[0],s[1]])
    #   # vals.append(0)



    # if s[0] + 1 <= self.env.rows - 1 and s[1] >= 0:

    #   vals.append(self.env.cur_values[s[0] + 1,s[1]])

    # else:

    #   vals.append(self.env.cur_values[s[0],s[1]])
    #   # vals.append(0)

    # return vals


  def improve_policy(self):

    for i in range(self.env.rows):

      for j in range(self.env.cols):

        s = (i,j)

        values = self.findMax(s)

        c = np.argmax(values)

        p = [self.agent.epsilon/4,self.agent.epsilon/4,self.agent.epsilon/4,self.agent.epsilon/4]

        p[c] = self.agent.epsilon/4 + 1 - self.agent.epsilon

        self.agent.policy_values[:,s[0],s[1]] = p


  def printPolicy(self):

    grid = np.zeros((self.env.rows,self.env.cols))    

    for i in range(self.env.rows):

      for j in range(self.env.cols):

        s = (i,j)

        grid[s] = np.argmax(self.agent.policy_values[:,s[0],s[1]])


    print(grid)

            



In [ ]:
e = GridWorld(4,4,(0,0),(3,3))

In [ ]:
ag = Agent(4,4,(0,0))

In [ ]:
p = PolicyIteration(ag, e)

In [ ]:
for i in range(100000):

  p.evaluate_policy()
  p.improve_policy()

In [ ]:
p.printPolicy()

[[3. 1. 1. 3.]
 [3. 3. 0. 0.]
 [1. 1. 3. 2.]
 [2. 1. 1. 0.]]


In [ ]:
val = []

for i in range(p.env.rows):

  for j in range(p.env.cols):

    val.append(max(p.env.qvalues[:,i,j]))


np.array([val]).reshape((4,-1))

array([[-1., -1., -1., -2.],
       [-2., -3., -2., -1.],
       [-2., -4., -5., -1.],
       [-1., -1., -6., -1.]])

In [ ]:
p.agent.policy_values

array([[[5.81395349e-07, 5.81395349e-07, 5.81395349e-07, 5.81395349e-07,
         5.81395349e-07],
        [5.81395349e-07, 5.81395349e-07, 5.81395349e-07, 5.81395349e-07,
         9.99998256e-01],
        [9.99998256e-01, 5.81395349e-07, 5.81395349e-07, 5.81395349e-07,
         5.81395349e-07],
        [5.81395349e-07, 5.81395349e-07, 5.81395349e-07, 5.81395349e-07,
         9.99998256e-01],
        [9.99998256e-01, 5.81395349e-07, 5.81395349e-07, 9.99998256e-01,
         5.81395349e-07]],

       [[5.81395349e-07, 9.99998256e-01, 5.81395349e-07, 9.99998256e-01,
         5.81395349e-07],
        [9.99998256e-01, 5.81395349e-07, 5.81395349e-07, 5.81395349e-07,
         5.81395349e-07],
        [5.81395349e-07, 9.99998256e-01, 9.99998256e-01, 9.99998256e-01,
         5.81395349e-07],
        [5.81395349e-07, 5.81395349e-07, 5.81395349e-07, 9.99998256e-01,
         5.81395349e-07],
        [5.81395349e-07, 5.81395349e-07, 5.81395349e-07, 5.81395349e-07,
         5.81395349e-07]],

      